In [ ]:
import IPython, graphviz
@IPython.core.magic.register_line_cell_magic
def dot(line, cell):
    return graphviz.Source(cell)

## For Starters

The basic idea of a starter is:

> - **pick a destination** in the _JupyterLab File Browser_
> - **click a button** in the _JupyterLab Launcher_
> - **see useful files**

A slightly more accurate version is:

> - configure via [traitlets](https://jupyter-notebook.readthedocs.io/en/stable/config.html) 
> - advertise to JupyterLab via the [REST API](./REST%20API.ipynb)
> - display in the _JupyterLab Launcher_
> - **click a button** in the _JupyterLab Launcher_
>   - or immediately start with a [Starter Tree URL](#Starter-Tree-URL)
> - zero or more (but usually one) times: 
>   - gather more information from the user via [react-jsonschema-form](https://react-jsonschema-form.readthedocs.io)
>   - perform further processing
>   - copy files via the [Contents API](https://jupyter-notebook.readthedocs.io/en/stable/extending/contents.html)
>   - **see useful files** in the _JupyterLab File Browser_
>   - run _JupyterLab Commands_ to do other things to JupyterLab

Which of these steps a particular starter performs depends primarily on its type.

### Types of Starters

#### Copy

> `"type": "copy"`
>
> `"src": "<an absolute or relative path>"`

The simplest starter, `copy`, just... copies. It can copy a single file, or a directory of files (and subdirectories). The `src` attribute tells the starter where to get the files.

In [ ]:
%%dot
digraph g { compound=true layout=dot rankdir=TB
    node[shape=none fontname="sans-serif"]
    graph[fontname="sans-serif" fontcolor="grey" color="none" fillcolor="#eeeeee" style=filled]
    label="a notional execution of a copy starter"
    subgraph cluster_files { label="Your Files"
        files
    }
    
    subgraph cluster_server { label="Notebook Server"
        get[label="/starters" fontname=monospace]
        post[label="/starters/{:name}/{:path}" fontname=monospace]
        contents
    }

    subgraph cluster_lab { label="JupyterLab"
        launcher
    }
    
    get -> launcher[label=①]
    launcher -> post[label=②]
    post -> contents[label=③]
    contents -> files[label=④]
    files -> contents[label=⑤]
    contents -> post[label=⑥]
    post -> launcher[label=⑦]
    launcher -> launcher[label=⑧]
}

`copy`, like all the starters, makes use of the [Contents API](https://jupyter-notebook.readthedocs.io/en/stable/extending/contents.html) directly. Existing files will _not_ be overwritten.

#### Python

> `"type": "python"`
>
> `"callable": "<a dotted notation python function>"`

A Python Starter is a function. This type has the fewest limitations, as it has full access to the `StarterManager` (and by extension, it's `parent`, the `NotebookApp`). This powers both the [Cookiecutter](#Cookiecutter) the [Notebook](#Notebook) starters, with the latter directly using the notebook server's _Kernel Manager_ to start short-lifespan kernels. 

In [ ]:
%%dot
digraph g { compound=true layout=dot rankdir=TB
    node[shape=none fontname="sans-serif"]
    graph[fontname="sans-serif" fontcolor="grey" color="none" fillcolor="#eeeeee" style=filled]
    label="a notional execution of a python starter"
    subgraph cluster_files { label="Your Files"
        files
    }
    
    subgraph cluster_server { label="Notebook Server"
        get[label="/starters" fontname=monospace]
        post[label="/starters/{:name}/{:path}" fontname=monospace]
        contents
        callable             
    }

    subgraph cluster_lab { label="JupyterLab"
        launcher
    }
    
    get -> launcher[label=①]
    launcher -> post[label=②]
    post -> callable[label=③]
    callable -> contents[label=④]
    contents -> files[label=⑤]
    files -> contents[label=⑥]
    contents -> callable[label=⑦]
    callable -> post[label=⑧]
    post -> launcher[label=⑨]
    launcher -> launcher[label=⑩]
}

#### Notebook

> `"type": "notebook"`

A notebook can be a starter. Each starter run gets its own, private kernel which can persist between interactions with the user. Communication with the server manager is handled through manipulating a copy of the notebook, specfically the notebook metadata. The advantages of this approach over the Python starter is:

- works with **any installed kernel**
- **state is maintained** between successive re-executions
- `jupyterlab-starters` provides **authoring support** for editing and validating the starter

In [ ]:
%%dot
digraph g { compound=true layout=dot rankdir=TB title="woooo"
    node[shape=none fontname="sans-serif"]
    graph[fontname="sans-serif" fontcolor="grey" color="none" fillcolor="#eeeeee" style=filled]
    label="a notional execution of a notebook starter"
    subgraph cluster_files { label="Your Files"
        files
    }
    
    subgraph cluster_server { label="Notebook Server"
        get[label="/starters" fontname=monospace]
        post[label="/starters/cookiecutter/{:path}" fontname=monospace]
        contents
        kernel
        tmpdir
    }

    subgraph cluster_lab { label="JupyterLab"
        launcher
        form1[label="initial form"]
        form2[label="dynamic form"]
    }

    get -> launcher[label=①]
    launcher -> form1[label=②]
    form1 -> post[label=③]
    post -> tmpdir[label=④]
    tmpdir -> post[label=⑤]
    tmpdir -> kernel[label=⑥]
    kernel -> tmpdir[label=⑦]
    post -> form2[label=⑧]
    form2 -> post[label=⑨]
    post -> tmpdir[label=⑩]
    tmpdir -> kernel[label=⑪]
    kernel -> tmpdir[label=⑫]
    tmpdir -> contents[label=⑬]
    contents -> files[label=⑭]
    files -> contents[label=⑮]
    contents -> post[label=⑯]
    post -> launcher[label=⑰]
    launcher -> launcher[label=⑲]
}

### Built-ins

#### Cookiecutter
The cookiecutter starter will be available if `cookiecutter` is [installed](./Users.ipynb#Cookiecutter) in the same Python environment as the `notebook` server.

> Additionally, if available, `importlib_metadata` will be used to list the (previously) curated list of community-contributed cookiecutters. It is now recommended to search for them directly on GitHub by [topic](https://github.com/topics/cookiecutter-template) or [advanced search](https://github.com/search?utf8=%E2%9C%93&q=path%3A%2F+filename%3Acookiecutter.json).

One of the original motivations for _Jupyter Starters_ was a way to provide a convenient, consistent, web-based experience for the [cookiecutter](https://cookiecutter.rtfd.io) ecosystem. Briefly, a cookiecutter is:

> - a repository, zip archive, or directory that contains
>   - `cookiecutter.json`
>   - a (potentially nested) directory that uses [Jinja2](https://jinja.palletsprojects.com) to describe file names and contents

What they may lack in dynamism, the make up for in consistency and robustness. 

In [ ]:
%%dot
digraph g { compound=true layout=dot rankdir=TB title="woooo"
    node[shape=none fontname="sans-serif"]
    graph[fontname="sans-serif" fontcolor="grey" color="none" fillcolor="#eeeeee" style=filled]
    label="a notional execution of the cookiecutter starter"
    subgraph cluster_files { label="Your Files"
        files
    }
    
    subgraph cluster_server { label="Notebook Server"
        get[label="/starters" fontname=monospace]
        post[label="/starters/cookiecutter/{:path}" fontname=monospace]
        contents
        cookiecutter
    }

    subgraph cluster_lab { label="JupyterLab"
        launcher
        form1[label="template form"]
        form2[label="cookiecutter form"]
    }
    get -> launcher[label=①]
    launcher -> form1[label=②]
    form1 -> post[label=③]
    post -> cookiecutter[label=④]
    cookiecutter -> git[label=⑤]
    git -> cookiecutter[label=⑥]
    cookiecutter -> post[label=⑧]
    post -> form2[label=⑨]
    form2 -> post[label=⑩]
    post -> cookiecutter[label=⑪]
    cookiecutter -> contents[label=⑫]
    contents -> files[label=⑬]
    files -> contents[label=⑭]
    contents -> post[label=⑮]
    post -> launcher[label=⑯]
    launcher -> launcher[label=⑰]
}

Under the hood, the cookiecutter starter is implemented as a [Python starter](#Python), and can be seen as tutorial in how to create a starter from a complex piece of existing functionality.

### Extras

#### Starter Tree URL

By specifying a special URL when starting JupyterLab, you can immediately start a Starter, without requiring the launcher. The pattern is:

```
{:protocol}://{:host}:{:port}{:base-url}/lab/tree/starter/{:starter-name}{:starter-path}
```

For example:
```
http://localhost:8888/lab/tree/starter/cookiecutter
```

On [Binder](https://mybinder.org), this path is determined by the `urlpath` `GET` parameter, for example:

```
https://mybinder.org/v2/gh/deathbeds/jupyterlab-starters/master?urlpath=lab%2Ftree%2Fcookiecutter%2Fexamples
```